# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,266.88,42,0,4.12,US,1709328928
1,1,skelleftea,64.7507,20.9528,275.37,100,100,5.14,SE,1709329373
2,2,margaret river,-33.9500,115.0667,288.52,77,0,6.67,AU,1709328665
3,3,waterville,44.5520,-69.6317,274.72,44,0,5.66,US,1709329374
4,4,bikenibeu village,1.3673,173.1241,303.13,79,40,3.60,KI,1709328952


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:


# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[city_data_df['Cloudiness'] == 0]
ideal_cities_df = ideal_cities_df[ideal_cities_df['Wind Speed']  < 5]
ideal_cities_df = ideal_cities_df[ideal_cities_df['Humidity'] < 80]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,266.88,42,0,4.12,US,1709328928
34,34,valparaiso,-33.0393,-71.6273,295.28,64,0,4.12,CL,1709329351
42,42,nay pyi taw,19.7450,96.1297,291.68,31,0,0.91,MM,1709329392
76,76,bethel,41.3712,-73.4140,280.25,46,0,4.63,US,1709329196
124,124,lashio,22.9333,97.7500,284.31,56,0,0.65,MM,1709329433
135,135,martil,35.6166,-5.2752,285.13,66,0,4.63,MA,1709329438
143,143,broome,42.2506,-75.8330,282.36,41,0,4.12,US,1709329443
153,153,tura,25.5198,90.2201,288.96,40,0,1.00,IN,1709328944
186,186,woodward,36.4337,-99.3904,291.72,47,0,4.63,US,1709329478
208,208,cherry hill,39.9348,-75.0307,282.40,47,0,4.63,US,1709329253


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,kodiak,57.7900,-152.4072,266.88,42,0,4.12,US,1709328928,
34,34,valparaiso,-33.0393,-71.6273,295.28,64,0,4.12,CL,1709329351,
42,42,nay pyi taw,19.7450,96.1297,291.68,31,0,0.91,MM,1709329392,
76,76,bethel,41.3712,-73.4140,280.25,46,0,4.63,US,1709329196,
124,124,lashio,22.9333,97.7500,284.31,56,0,0.65,MM,1709329433,
135,135,martil,35.6166,-5.2752,285.13,66,0,4.63,MA,1709329438,
143,143,broome,42.2506,-75.8330,282.36,41,0,4.12,US,1709329443,
153,153,tura,25.5198,90.2201,288.96,40,0,1.00,IN,1709328944,
186,186,woodward,36.4337,-99.3904,291.72,47,0,4.63,US,1709329478,
208,208,cherry hill,39.9348,-75.0307,282.40,47,0,4.63,US,1709329253,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(15)

Starting hotel search
kodiak - nearest hotel: Shelikof Lodge
valparaiso - nearest hotel: Ibis Hotel
nay pyi taw - nearest hotel: နန်းရွှေပြည်အိမ်ခြံမြေ
bethel - nearest hotel: Hampton Inn Danbury
lashio - nearest hotel: Golden Kaniri Hotel
martil - nearest hotel: Residence Mediterrane
broome - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
woodward - nearest hotel: Sands Inn
cherry hill - nearest hotel: Days Inn
khovd - nearest hotel: Khovd hotel
kushalgarh - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
ghat - nearest hotel: فندق تاسيلي
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
emerald - nearest hotel: Emerald Central Hotel
ampanihy - nearest hotel: Hotel restaurant Angora
nizwa - nearest hotel: Nizwa Heritage Inn
chiredzi - nearest hotel: Nesbitt Arms Hotel
callao - nearest hotel: Casa Ronald
selebi-phikwe - nearest hotel: Cresta Bosele Hotel
ben gardane - nearest hotel: No hotel 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,kodiak,57.7900,-152.4072,266.88,42,0,4.12,US,1709328928,Shelikof Lodge
34,34,valparaiso,-33.0393,-71.6273,295.28,64,0,4.12,CL,1709329351,Ibis Hotel
42,42,nay pyi taw,19.7450,96.1297,291.68,31,0,0.91,MM,1709329392,နန်းရွှေပြည်အိမ်ခြံမြေ
76,76,bethel,41.3712,-73.4140,280.25,46,0,4.63,US,1709329196,Hampton Inn Danbury
124,124,lashio,22.9333,97.7500,284.31,56,0,0.65,MM,1709329433,Golden Kaniri Hotel
135,135,martil,35.6166,-5.2752,285.13,66,0,4.63,MA,1709329438,Residence Mediterrane
143,143,broome,42.2506,-75.8330,282.36,41,0,4.12,US,1709329443,No hotel found
153,153,tura,25.5198,90.2201,288.96,40,0,1.00,IN,1709328944,No hotel found
186,186,woodward,36.4337,-99.3904,291.72,47,0,4.63,US,1709329478,Sands Inn
208,208,cherry hill,39.9348,-75.0307,282.40,47,0,4.63,US,1709329253,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ['Hotel Name', 'City']
)
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)